In [1]:
from superpipe import *
import pandas as pd
from pydantic import BaseModel, Field

In [2]:
joke_prompt = lambda row: f"""
Tell me a joke about {row['topic']}
"""

JokesStep = steps.SimpleLLMStep(
  prompt=joke_prompt,
  model=models.gpt35,
  name="joke"
)

In [3]:
topics = ['Beets', 'Bears', 'Battlestar Gallactica']

In [4]:
jokes_df = pd.DataFrame(topics)
jokes_df.columns = ["topic"]

In [5]:
jokes_df

topic
0                  Beets
1                  Bears
2  Battlestar Gallactica

In [23]:
def evaluate_prompt(row):
    return f"""
    Is the following joke pretty funny? Your bar should be making a friend laugh
    {row['joke']}

    Return a json object with a single boolean key called 'evaluation'
    """

def evaluate_joke(row):
    return llm.get_structured_llm_response(evaluate_prompt(row), models.gpt4).content['evaluation']

In [24]:
comedian = pipeline.Pipeline([
    JokesStep,
], evaluation_fn=evaluate_joke)

comedian.apply(jokes_df)

Running step joke...


100%|██████████| 3/3 [00:02<00:00,  1.43it/s]


0     True
1     True
2    False
dtype: bool


topic                                           __joke__  \
0                  Beets  {'input_tokens': 16, 'output_tokens': 23, 'inp...   
1                  Bears  {'input_tokens': 15, 'output_tokens': 25, 'inp...   
2  Battlestar Gallactica  {'input_tokens': 19, 'output_tokens': 22, 'inp...   

                                                joke  
0  Why did the beet break up with the brussels sp...  
1  Why did the bear bring a flashlight to the par...  
2  Why did the Cylon break up with his girlfriend...

In [25]:
comedian.statistics

PipelineStatistics(score=0.6666666666666666, input_tokens=defaultdict(<class 'int'>, {}), output_tokens=defaultdict(<class 'int'>, {}), input_cost=0.0, output_cost=0.0, num_success=0, num_failure=0, total_latency=0.0)

In [22]:
jokes_df.joke.values

array(['Why did the beet turn red?\n\nBecause it saw the salad dressing!',
       "Why don't bears like fast food? Because they can never catch it!",
       'Why did the Cylon break up with his toaster girlfriend? Because she kept burning his bread!'],
      dtype=object)

In [29]:
JokesStep4 = steps.SimpleLLMStep(
  prompt=joke_prompt,
  model=models.gpt4,
  name="joke4"
)

In [48]:
def evaluate_side_by_side_prompt(row):
    return f"""
    You are given two jokes. Rate which is funnier:
    joke 1: {row['joke']}
    joke 2: {row['joke4']}

    If joke 1 is funnier or they are similar, return false. If joke 2 is funnier return true.

    Return a json object with a single boolean key called 'evaluation'
    """

def evaluate_side_by_side(row):
    return llm.get_structured_llm_response(evaluate_side_by_side_prompt(row), models.gpt4).content['evaluation']

In [52]:
jokes_df[['joke', 'joke4']].values

array([['Why did the beet blush?\n\nBecause it saw the salad dressing!',
        'Why did the beet turn red?\n\nBecause it saw the salad dressing!'],
       ["Why don't bears wear socks?\n\nBecause they have bear feet!",
        "Why don't bears wear socks?\n\nBecause they like to walk bear-foot!"],
       ['Why did the Cylon go to therapy?\n\nTo work on their "toasters" issues!',
        'Why did the Cylon buy an iPhone?\n\nBecause he heard it comes with Siri-usly good voice recognition!']],
      dtype=object)

In [50]:
comedian = pipeline.Pipeline([
    JokesStep,
    JokesStep4,
], evaluation_fn=evaluate_side_by_side)

comedian.apply(jokes_df)

Running step joke...


100%|██████████| 3/3 [00:01<00:00,  1.95it/s]


Running step joke4...


100%|██████████| 3/3 [00:02<00:00,  1.06it/s]


0    False
1    False
2     True
dtype: bool


topic                                           __joke__  \
0                  Beets  {'input_tokens': 16, 'output_tokens': 13, 'inp...   
1                  Bears  {'input_tokens': 15, 'output_tokens': 13, 'inp...   
2  Battlestar Gallactica  {'input_tokens': 19, 'output_tokens': 19, 'inp...   

                                                joke  \
0  Why did the beet blush?\n\nBecause it saw the ...   
1  Why don't bears wear socks?\n\nBecause they ha...   
2  Why did the Cylon go to therapy?\n\nTo work on...   

                                           __joke4__  \
0  {'input_tokens': 16, 'output_tokens': 14, 'inp...   
1  {'input_tokens': 15, 'output_tokens': 15, 'inp...   
2  {'input_tokens': 19, 'output_tokens': 22, 'inp...   

                                               joke4  
0  Why did the beet turn red?\n\nBecause it saw t...  
1  Why don't bears wear socks?\n\nBecause they li...  
2  Why did the Cylon buy an iPhone?\n\nBecause he...